In [127]:
import pandas as pd
import numpy as np

from scipy.integrate import quad
from scipy.optimize import minimize


from multiprocessing import Pool
from functools import partial
import time
from datetime import datetime

from typing import Dict

import json
import logging
import sys

In [128]:
sample = json.load(open('data/snls_snia.json'))

df = pd.DataFrame(sample)

In [129]:
def Dc(z, Omega_l, Omega_m, Omega_k, w):
    def f(z, Omega_l, Omega_m, Omega_k, w):
        return (np.sqrt(Omega_l*(1 + z)**(-3 - 3*w) + Omega_m*(1 + z)**3 + Omega_k*(1 + z)**2 + 9.2e-5*(1 + z)**4))**(-1)
    return quad(f, 0, z, args = (Omega_l, Omega_m, Omega_k, w))[0]

def Dt(z, Omega_l, Omega_m, Omega_k, w):
    if Omega_k > 0:
        return np.sin(np.sqrt(Omega_k)*Dc(z, Omega_l, Omega_m, Omega_k, w))/np.sqrt(Omega_k)
    elif Omega_k == 0:
        return Dc(z, Omega_l, Omega_m, Omega_k, w)
    elif Omega_k:
        return np.sinh(np.sqrt(abs(Omega_k))*Dc(z, Omega_l, Omega_m, Omega_k, w))/np.sqrt(abs(Omega_k))

def Dl(z, Omega_l, Omega_m, Omega_k, w):
    return (1 + z) * Dt(z, Omega_l, Omega_m, Omega_k, w)
               
def mu(z, Omega_l, Omega_m, Omega_k, w, Hubble):
    c = 3e5 #(km/s)
    return 5*np.log10(Dl(z, Omega_l, Omega_m, Omega_k, w)) + 25 + 5*np.log10(c/ Hubble)

def chi2_i(mu_o, sigma_o, z_obs, Omega_l, Omega_m, Omega_k, w, Hubble):
    return ((mu(z_obs, Omega_l, Omega_m, Omega_k, w, Hubble) - mu_o)**2)/sigma_o**2

def chi2(Omega_l, Omega_m, Omega_k, w, Hubble, df): 
    values = []
    for idx in df.index:
        values.append(chi2_i(df.mu[idx], df.sigma[idx], df.z[idx], Omega_l, Omega_m, Omega_k, w, Hubble))
    return sum(values)

def ratio(Omega_l, Omega_m, Omega_k, w, Hubble, df, chi2null):
    return chi2(Omega_l, Omega_m, Omega_k, w, Hubble, df) - chi2null

In [130]:
def h(x, fixed, df):
    """
    This is a wrapper for the chi2 function. 
    A workaround to the fact that scipy.optimize.minimize uses positional arguments
    
        Given a dict of fixed variables, this fuction enables minimize to pass an array of arguments
    """
    
    full_parameters = ['Omega_l', 'Omega_m', 'Omega_k', 'w',  'Hubble']
    variable_parameters = []
    
    for param in full_parameters:
        if param not in list(fixed.keys()):
            variable_parameters.append(param)
            
    _vars = dict(zip(variable_parameters, x)) # Creates dictionary of variables to be optimized
    
    
    fixed['df'] = df
    g = partial(chi2, **fixed) # Creates partial function fixing variables set by `fixed`
        
    # Unzips `_vars` as named args to `g`
    return g(**_vars) 

In [131]:
# Optimizing every variable to find null hypothesis

fixed = {}

x0 = [0.74, 0.24, 0.02, -1, 71] #['Omega_l', 'Omega_m', 'Omega_k', 'w',  'Hubble']

minimum = minimize(h, x0 = x0, args = (fixed, df), method = 'Nelder-Mead', tol = 1e-6, options = {'maxiter': 10000})

In [132]:
minimum

 final_simplex: (array([[ 3.90338078e-01,  1.47505529e-01,  2.51625917e-02,
        -9.18027648e-01,  9.34891944e+01],
       [ 3.90338082e-01,  1.47505531e-01,  2.51625919e-02,
        -9.18027647e-01,  9.34891940e+01],
       [ 3.90338082e-01,  1.47505530e-01,  2.51625919e-02,
        -9.18027648e-01,  9.34891940e+01],
       [ 3.90338079e-01,  1.47505530e-01,  2.51625918e-02,
        -9.18027648e-01,  9.34891943e+01],
       [ 3.90338084e-01,  1.47505531e-01,  2.51625921e-02,
        -9.18027647e-01,  9.34891937e+01],
       [ 3.90338077e-01,  1.47505529e-01,  2.51625916e-02,
        -9.18027648e-01,  9.34891945e+01]]), array([110.96996089, 110.96996089, 110.96996089, 110.96996089,
       110.96996089, 110.96996089]))
           fun: 110.9699608910798
       message: 'Optimization terminated successfully.'
          nfev: 3181
           nit: 1995
        status: 0
       success: True
             x: array([ 3.90338078e-01,  1.47505529e-01,  2.51625917e-02, -9.18027648e-01,
       